In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
d = {'Region':['issuer_region_name','approval_rate_this_year','approval_rate_last_year','delta'],
     'Issuer':['issuer_region_name','issuer_country_name','gpns_macro_segment','issuer_name'],
     'Acquirer':['issuer_region_name','issuer_country_name','gpns_macro_segment','acquirer_name'],
     'Merchant':['issuer_region_name','issuer_country_name','gpns_macro_segment','parent_aggregate_merchant_name'],
     'Issuer-Acquirer':['issuer_region_name','issuer_country_name','gpns_macro_segment','issuer_name','acquirer_name'],
     'Issuer-Merchant':['issuer_region_name','issuer_country_name','gpns_macro_segment','issuer_name','parent_aggregate_merchant_name'],
     'Acquirer-Merchant':['issuer_region_name','issuer_country_name','gpns_macro_segment','acquirer_name','parent_aggregate_merchant_name'],
     'Issuer-Acquirer-Merchant':['issuer_region_name','issuer_country_name','gpns_macro_segment','issuer_name','acquirer_name','parent_aggregate_merchant_name']
    }

In [19]:
data = {}
df = pd.DataFrame({
    'issuer_region_name':['NAM','LAC','EEMEA','ASIA/PACIFIC','EUROPE'],
    'approval_rate_this_year':[0.932,0.884,0.898,0.914,0.929],
    'approval_rate_last_year':[0.9369,0.88,0.875,0.901,0.92],
})
df['delta_ar'] = (df['approval_rate_this_year'] - df['approval_rate_last_year'])
data['Region'] = df
df = df.style.format({'approval_rate_this_year': '{:.2%}', 'approval_rate_last_year': '{:.2%}', 'delta_ar': '{:.3%}'})
df

,issuer_region_name,approval_rate_this_year,approval_rate_last_year,delta_ar
0,NAM,93.20%,93.69%,-0.490%
1,LAC,88.40%,88.00%,0.400%
2,EEMEA,89.80%,87.50%,2.300%
3,ASIA/PACIFIC,91.40%,90.10%,1.300%
4,EUROPE,92.90%,92.00%,0.900%


In [24]:
import numpy as np
import pandas as pd

np.random.seed(42)

regions = ['NAM', 'EEMEA', 'ASIA/PACIFIC', 'EUROPE', 'LAC']
countries = ['USA', 'ITA', 'AUS', 'CAN', 'EGY', 'UK', 'BRA', 'IND', 'DEU']
segments = ['COMMERCIAL_CREDIT', 'COMMERCIAL_DEBIT',
            'CONSUMER_CREDIT', 'CONSUMER_DEBIT', 'PREPAID']

issuers = [
    'CITIBANK, N.A.',
    'BANCO SANTANDER',
    'HSBC BANK PLC',
    'JPMORGAN CHASE BANK',
    'BARCLAYS BANK',
    'ICICI BANK',
    'BANK OF AMERICA',
    'DEUTSCHE BANK AG'
]

rows = []

for _ in range(10000):
    for issuer in issuers:
        
        region = np.random.choice(regions)
        country = np.random.choice(countries)
        segment = np.random.choice(segments)
        
        # Base monthly volume (issuer-specific)
        base_volume = np.random.randint(50_000, 200_000)
        
        # Realistic approval rate between 75% - 95%
        base_approval_rate = np.random.uniform(0.75, 0.95)
        
        approvals = int(base_volume * base_approval_rate)
        declines = base_volume - approvals
        
        # MoM variation (±5%)
        mom_factor = np.random.uniform(0.95, 1.05)
        approvals_mom = int(approvals * mom_factor)
        declines_mom = int(declines * mom_factor)
        
        # YoY variation (±12%)
        yoy_factor = np.random.uniform(0.88, 1.12)
        approvals_yoy = int(approvals * yoy_factor)
        declines_yoy = int(declines * yoy_factor)
        
        rows.append([
            region, country, segment, issuer,
            approvals, declines,
            approvals_mom, declines_mom,
            approvals_yoy, declines_yoy
        ])

columns = [
    'issuer_region_name',
    'issuer_country_name',
    'gpns_macro_segment',
    'issuer_name',
    'issuer_approvals',
    'issuer_declines',
    'issuer_approvals_mom',
    'issuer_declines_mom',
    'issuer_approvals_yoy',
    'issuer_declines_yoy'
]

df = pd.DataFrame(rows, columns=columns)

# Calculate approval rates
df['issuer_approval_rate'] = (
    df['issuer_approvals'] /
    (df['issuer_approvals'] + df['issuer_declines'])
).astype('float32')

df['issuer_approval_rate_mom'] = (
    df['issuer_approvals_mom'] /
    (df['issuer_approvals_mom'] + df['issuer_declines_mom'])
).astype('float32')

df['issuer_approval_rate_yoy'] = (
    df['issuer_approvals_yoy'] /
    (df['issuer_approvals_yoy'] + df['issuer_declines_yoy'])
).astype('float32')

# Convert numeric columns to int32
int_cols = [
    'issuer_approvals', 'issuer_declines',
    'issuer_approvals_mom', 'issuer_declines_mom',
    'issuer_approvals_yoy', 'issuer_declines_yoy'
]

df[int_cols] = df[int_cols].astype('int32')
data['Issuer'] = df
df.head()


,issuer_region_name,issuer_country_name,gpns_macro_segment,issuer_name,issuer_approvals,issuer_declines,issuer_approvals_mom,issuer_declines_mom,issuer_approvals_yoy,issuer_declines_yoy,issuer_approval_rate,issuer_approval_rate_mom,issuer_approval_rate_yoy
0,EUROPE,IND,PREPAID,"CITIBANK, N.A.",88016,16870,84495,16195,87155,16705,0.839159,0.839160,0.839158
1,LAC,CAN,CONSUMER_CREDIT,BANCO SANTANDER,89238,27983,91219,28604,98630,30928,0.761280,0.761281,0.761281
2,EEMEA,EGY,COMMERCIAL_CREDIT,HSBC BANK PLC,115679,19626,114891,19492,109882,18642,0.854950,0.854952,0.854953
3,ASIA/PACIFIC,AUS,CONSUMER_DEBIT,JPMORGAN CHASE BANK,101183,5703,100846,5684,109923,6195,0.946644,0.946644,0.946649
4,LAC,DEU,COMMERCIAL_DEBIT,BARCLAYS BANK,179898,11801,181036,11875,174950,11476,0.938440,0.938443,0.938442


In [26]:
regions = ['NAM', 'EEMEA', 'ASIA/PACIFIC', 'EUROPE', 'LAC']
countries = ['USA', 'ITA', 'AUS', 'CAN', 'EGY', 'UK', 'BRA', 'IND', 'DEU']
segments = ['COMMERCIAL_CREDIT', 'COMMERCIAL_DEBIT',
            'CONSUMER_CREDIT', 'CONSUMER_DEBIT', 'PREPAID']

acquirers = [
    'CITIBANK, N.A.',
    'BANCO SANTANDER',
    'HSBC BANK PLC',
    'JPMORGAN CHASE BANK',
    'BARCLAYS BANK',
    'ICICI BANK',
    'BANK OF AMERICA',
    'DEUTSCHE BANK AG'
]

rows = []

for _ in range(10000):
    for issuer in acquirers:
        
        region = np.random.choice(regions)
        country = np.random.choice(countries)
        segment = np.random.choice(segments)
        
        # Base monthly volume (issuer-specific)
        base_volume = np.random.randint(50_000, 200_000)
        
        # Realistic approval rate between 75% - 95%
        base_approval_rate = np.random.uniform(0.75, 0.95)
        
        approvals = int(base_volume * base_approval_rate)
        declines = base_volume - approvals
        
        # MoM variation (±5%)
        mom_factor = np.random.uniform(0.95, 1.05)
        approvals_mom = int(approvals * mom_factor)
        declines_mom = int(declines * mom_factor)
        
        # YoY variation (±12%)
        yoy_factor = np.random.uniform(0.88, 1.12)
        approvals_yoy = int(approvals * yoy_factor)
        declines_yoy = int(declines * yoy_factor)
        
        rows.append([
            region, country, segment, issuer,
            approvals, declines,
            approvals_mom, declines_mom,
            approvals_yoy, declines_yoy
        ])

columns = [
    'issuer_region_name',
    'issuer_country_name',
    'gpns_macro_segment',
    'acquirer_name',
    'acquirer_approvals',
    'acquirer_declines',
    'acquirer_approvals_mom',
    'acquirer_declines_mom',
    'acquirer_approvals_yoy',
    'acquirer_declines_yoy'
]

df = pd.DataFrame(rows, columns=columns)

# Calculate approval rates
df['acquirer_approval_rate'] = (
    df['acquirer_approvals'] /
    (df['acquirer_approvals'] + df['acquirer_declines'])
).astype('float32')

df['acquirer_approval_rate_mom'] = (
    df['acquirer_approvals_mom'] /
    (df['acquirer_approvals_mom'] + df['acquirer_declines_mom'])
).astype('float32')

df['acquirer_approval_rate_yoy'] = (
    df['acquirer_approvals_yoy'] /
    (df['acquirer_approvals_yoy'] + df['acquirer_declines_yoy'])
).astype('float32')

# Convert numeric columns to int32
int_cols = [
    'acquirer_approvals', 'acquirer_declines',
    'acquirer_approvals_mom', 'acquirer_declines_mom',
    'acquirer_approvals_yoy', 'acquirer_declines_yoy'
]

df[int_cols] = df[int_cols].astype('int32')
data['Acquirer'] = df
df.head()


,issuer_region_name,issuer_country_name,gpns_macro_segment,acquirer_name,acquirer_approvals,acquirer_declines,acquirer_approvals_mom,acquirer_declines_mom,acquirer_approvals_yoy,acquirer_declines_yoy,acquirer_approval_rate,acquirer_approval_rate_mom,acquirer_approval_rate_yoy
0,ASIA/PACIFIC,BRA,CONSUMER_CREDIT,"CITIBANK, N.A.",115634,8798,117416,8933,101879,7751,0.929295,0.929299,0.929299
1,EUROPE,IND,CONSUMER_CREDIT,BANCO SANTANDER,62265,11275,60483,10952,68173,12344,0.846682,0.846686,0.846691
2,EEMEA,EGY,CONSUMER_CREDIT,HSBC BANK PLC,160101,16690,156569,16321,151611,15804,0.905595,0.905599,0.905600
3,EUROPE,IND,CONSUMER_CREDIT,JPMORGAN CHASE BANK,41875,8896,40919,8692,44079,9364,0.824782,0.824797,0.824785
4,EUROPE,EGY,CONSUMER_DEBIT,BARCLAYS BANK,92747,20049,96892,20945,92890,20079,0.822254,0.822254,0.822261


In [34]:
# writer = pd.ExcelWriter('global_data_202512_v4.xlsx', engine='openpyxl')
with pd.ExcelWriter('global_data_202512_v4.xlsx') as writer:
    for i, frame in data.items():
        print(i)
        frame.to_excel(writer, sheet_name = i, index=False)

Region
Issuer
Acquirer


In [35]:
for sheet in ['Region','Issuer','Acquirer']:
    print(pd.read_excel("global_data_202512_v4.xlsx",sheet_name=sheet, engine='openpyxl').head(3))

  issuer_region_name  approval_rate_this_year  approval_rate_last_year  \
0                NAM                    0.932                   0.9369   
1                LAC                    0.884                   0.8800   
2              EEMEA                    0.898                   0.8750   

   delta_ar  
0   -0.0049  
1    0.0040  
2    0.0230  
  issuer_region_name issuer_country_name gpns_macro_segment      issuer_name  \
0             EUROPE                 IND            PREPAID   CITIBANK, N.A.   
1                LAC                 CAN    CONSUMER_CREDIT  BANCO SANTANDER   
2              EEMEA                 EGY  COMMERCIAL_CREDIT    HSBC BANK PLC   

   issuer_approvals  issuer_declines  issuer_approvals_mom  \
0             88016            16870                 84495   
1             89238            27983                 91219   
2            115679            19626                114891   

   issuer_declines_mom  issuer_approvals_yoy  issuer_declines_yoy  \
0       

In [31]:
excel = pd.ExcelFile('global_data_202512_v4.xlsx', engine='openpyxl')

BadZipFile: File is not a zip file